# Bridge Pledge

### Import supporting Libaries

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

### Metadata - Base Table

In [3]:
meta_data = pd.read_excel('/kaggle/input/bridge-pledge-model-data/Model-Senate-Beta-1.1.xlsx')
#meta_data.head(5)
meta_data = meta_data.loc[:, ['bioguide_id', 'first name','last name', 'full name', 'party name', 'party letter','district','state','status']]
meta_data.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election


### Source Data 

In [4]:
# Load and filter the source file
source_A = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Senate-GovTrack-Author-Bill-with-Cosponsor.xlsx')
source_A = source_A[['bioguide_id', 'bills-with-cosponsor-other-party']]

# Merge source_A with meta_data on 'bioguide_id'
final = pd.merge(meta_data, source_A, left_on='bioguide_id', right_on='bioguide_id', how='left')

# Fill missing values in 'bills-with-cosponsor-other-party' with the column mean
mean_A = round(source_A['bills-with-cosponsor-other-party'].mean(), 2)
final['bills-with-cosponsor-other-party'] = final['bills-with-cosponsor-other-party'].fillna(mean_A)

# Calculate mean and standard deviation for normalization
mean_A = round(final['bills-with-cosponsor-other-party'].mean(), 2)
std_A = round(final['bills-with-cosponsor-other-party'].std(), 2)

# Normalize the 'bills-with-cosponsor-other-party' column
final['norm_A'] = norm.cdf(final['bills-with-cosponsor-other-party'], mean_A, std_A) * 100

# Remove duplicates based on 'bioguide_id'
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Preview the final DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,norm_A
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,44.013788
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,47.795341
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,49.980985
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,10.621675
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,31.409687


### Source Data 

In [5]:
# Load and filter the source file
source_B = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Senate-CGS-2024.07.xlsx')
source_B = source_B[['bioguide', 'cg_official_performance']]

# Merge source_B with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_B, left_on='bioguide_id', right_on='bioguide', how='left')

# Fill missing values in 'cg_official_performance' with the column mean
mean_B = round(source_B['cg_official_performance'].mean(), 2)
final['cg_official_performance'] = final['cg_official_performance'].fillna(mean_B)

# Drop the 'biocode' column after merging
final.drop('bioguide', axis=1, inplace=True)

# Calculate mean and standard deviation for normalization
mean_B = round(final['cg_official_performance'].mean(), 2)
std_B = round(final['cg_official_performance'].std(), 2)

# Normalize the 'cg_official_performance' column
final['norm_B'] = norm.cdf(final['cg_official_performance'], mean_B, std_B) * 100

# Remove duplicates based on 'bioguide_id'
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Preview the final DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,norm_A,cg_official_performance,norm_B
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,44.013788,25.0,96.956362
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,47.795341,7.0,41.347901
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,49.980985,0.0,15.090536
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,10.621675,0.0,15.090536
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,31.409687,16.0,79.613841


In [6]:
# Load and filter the source file
source_D = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Senate-Lugar-Bipartisan.xlsx')
source_D = source_D[['bioguide', 'lugar_score']]

# Merge source_D with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_D, left_on='bioguide_id', right_on='bioguide', how='left')

# Fill missing values in 'lugar_score' with the column mean
mean_D = round(source_D['lugar_score'].mean(), 2)
final['lugar_score'] = final['lugar_score'].fillna(mean_D)

# Drop the 'bioguide' column after merging
final.drop('bioguide', axis=1, inplace=True)

# Calculate mean and standard deviation for normalization
mean_D = round(final['lugar_score'].mean(), 2)
std_D = round(final['lugar_score'].std(), 2)

# Normalize the 'lugar_score' column
final['norm_D'] = norm.cdf(final['lugar_score'], mean_D, std_D) * 100

# Remove duplicates based on 'bioguide_id'
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Preview the final DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,norm_A,cg_official_performance,norm_B,lugar_score,norm_D
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,44.013788,25.0,96.956362,1.53979,95.943886
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,47.795341,7.0,41.347901,-0.14432,44.946141
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,49.980985,0.0,15.090536,-1.66551,3.459070
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,10.621675,0.0,15.090536,-1.25831,8.615971
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,31.409687,16.0,79.613841,0.67637,78.373087


### Source Data 

In [7]:
# Load and filter the source file
source_E = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Senate-GovTrack-Cosponsor-others-bills.xlsx')
source_E = source_E[['bioguide_id', 'cosponsored-other-party']]

# Merge source_E with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_E, left_on='bioguide_id', right_on='bioguide_id', how='left')

# Fill missing values in 'cosponsored-other-party' with the column mean
mean_E = round(source_E['cosponsored-other-party'].mean(), 2)
final['cosponsored-other-party'] = final['cosponsored-other-party'].fillna(mean_E)

# Calculate mean and standard deviation for normalization
mean_E = round(final['cosponsored-other-party'].mean(), 2)
std_E = round(final['cosponsored-other-party'].std(), 2)

# Normalize the 'cosponsored-other-party' column
final['norm_E'] = norm.cdf(final['cosponsored-other-party'], mean_E, std_E) * 100

# Remove duplicates based on 'bioguide_id'
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,norm_A,cg_official_performance,norm_B,lugar_score,norm_D,cosponsored-other-party,norm_E
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,44.013788,25.0,96.956362,1.53979,95.943886,63.606557,99.907021
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,47.795341,7.0,41.347901,-0.14432,44.946141,35.555556,73.760246
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,49.980985,0.0,15.090536,-1.66551,3.459070,28.790000,51.548901
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,10.621675,0.0,15.090536,-1.25831,8.615971,17.241379,16.342936
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,31.409687,16.0,79.613841,0.67637,78.373087,40.399002,85.621332


### Source Data 

In [8]:
# Load and filter the source file
source_F1 = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Both-PRL.xlsx')
source_F1 = source_F1[['bioguide_id', 'communication_outcome_bipartisanship_sum']]

# Merge source_F1 with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_F1, left_on='bioguide_id', right_on='bioguide_id', how='left')

# Fill missing values in 'communication_outcome_bipartisanship_sum' with the column mean
mean_F1 = round(source_F1['communication_outcome_bipartisanship_sum'].mean(), 2)
final['communication_outcome_bipartisanship_sum'] = final['communication_outcome_bipartisanship_sum'].fillna(mean_F1)

# Calculate mean and standard deviation for normalization
mean_F1 = round(final['communication_outcome_bipartisanship_sum'].mean(), 2)
std_F1 = round(final['communication_outcome_bipartisanship_sum'].std(), 2)

# Normalize the 'communication_outcome_bipartisanship_sum' column
final['norm_F1'] = norm.cdf(final['communication_outcome_bipartisanship_sum'], mean_F1, std_F1) * 100

# Remove duplicates based on 'bioguide_id'
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,norm_A,cg_official_performance,norm_B,lugar_score,norm_D,cosponsored-other-party,norm_E,communication_outcome_bipartisanship_sum,norm_F1
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,44.013788,25.0,96.956362,1.53979,95.943886,63.606557,99.907021,197,34.092129
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,47.795341,7.0,41.347901,-0.14432,44.946141,35.555556,73.760246,274,47.341407
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,49.980985,0.0,15.090536,-1.66551,3.459070,28.790000,51.548901,185,32.152269
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,10.621675,0.0,15.090536,-1.25831,8.615971,17.241379,16.342936,160,28.268183
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,31.409687,16.0,79.613841,0.67637,78.373087,40.399002,85.621332,164,28.874276


### Source Data 

In [9]:
# Load and filter the source file
source_F2 = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Both-PRL.xlsx')
source_F2 = source_F2[['bioguide_id', 'communication_outcome_bipartisanship_mean']]

# Merge source_F2 with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_F2, left_on='bioguide_id', right_on='bioguide_id', how='left')

# Fill missing values in 'communication_outcome_bipartisanship_mean' with the column mean
mean_F2 = round(source_F2['communication_outcome_bipartisanship_mean'].mean(), 2)
final['communication_outcome_bipartisanship_mean'] = final['communication_outcome_bipartisanship_mean'].fillna(mean_F2)

# Calculate mean and standard deviation for normalization
mean_F2 = round(final['communication_outcome_bipartisanship_mean'].mean(), 2)
std_F2 = round(final['communication_outcome_bipartisanship_mean'].std(), 2)

# Normalize the 'communication_outcome_bipartisanship_mean' column
final['norm_F2'] = norm.cdf(final['communication_outcome_bipartisanship_mean'], mean_F2, std_F2) * 100

# Remove duplicates based on 'bioguide_id'
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,...,cg_official_performance,norm_B,lugar_score,norm_D,cosponsored-other-party,norm_E,communication_outcome_bipartisanship_sum,norm_F1,communication_outcome_bipartisanship_mean,norm_F2
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,...,25.0,96.956362,1.53979,95.943886,63.606557,99.907021,197,34.092129,5.98,36.098041
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,...,7.0,41.347901,-0.14432,44.946141,35.555556,73.760246,274,47.341407,4.83,28.580110
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,...,0.0,15.090536,-1.66551,3.459070,28.790000,51.548901,185,32.152269,9.74,62.941032
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,...,0.0,15.090536,-1.25831,8.615971,17.241379,16.342936,160,28.268183,2.49,16.042791
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,...,16.0,79.613841,0.67637,78.373087,40.399002,85.621332,164,28.874276,6.65,40.765683


### Source Data 

In [10]:
# Load and filter the source file
source_G = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Senate-CGS-2024.07.xlsx')
source_G = source_G[['bioguide', 'cg_bonus']]

# Merge source_G with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_G, left_on='bioguide_id', right_on='bioguide', how='left')

# Fill missing values in 'cg_bonus' with the column mean
mean_G = round(source_G['cg_bonus'].mean(), 2)
final['cg_bonus'] = final['cg_bonus'].fillna(mean_G)

# Drop 'biocode' column after merge
final.drop('bioguide', axis=1, inplace=True)

# Calculate mean and standard deviation for normalization
mean_G = round(final['cg_bonus'].mean(), 2)
std_G = round(final['cg_bonus'].std(), 2)

# Normalize the 'cg_bonus' column
final['norm_G'] = norm.cdf(final['cg_bonus'], mean_G, std_G) * 100

# Remove duplicates based on 'bioguide_id'
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,...,lugar_score,norm_D,cosponsored-other-party,norm_E,communication_outcome_bipartisanship_sum,norm_F1,communication_outcome_bipartisanship_mean,norm_F2,cg_bonus,norm_G
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,...,1.53979,95.943886,63.606557,99.907021,197,34.092129,5.98,36.098041,0.0,24.283001
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,...,-0.14432,44.946141,35.555556,73.760246,274,47.341407,4.83,28.580110,0.0,24.283001
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,...,-1.66551,3.459070,28.790000,51.548901,185,32.152269,9.74,62.941032,0.0,24.283001
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,...,-1.25831,8.615971,17.241379,16.342936,160,28.268183,2.49,16.042791,0.0,24.283001
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,...,0.67637,78.373087,40.399002,85.621332,164,28.874276,6.65,40.765683,0.0,24.283001


### Source Data 

In [11]:
# Load and filter the source file for cg_communications
source_H = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Senate-CGS-2024.07.xlsx')
source_H = source_H[['bioguide', 'cg_communications']]

# Merge source_H with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_H, left_on='bioguide_id', right_on='bioguide', how='left')

# Fill missing values in 'cg_communications' with the column mean
mean_H = round(source_H['cg_communications'].mean(), 2)
final['cg_communications'] = final['cg_communications'].fillna(mean_H)

# Drop 'biocode' column after merge
final.drop('bioguide', axis=1, inplace=True)

# Calculate mean and standard deviation for normalization
mean_H = round(final['cg_communications'].mean(), 2)
std_H = round(final['cg_communications'].std(), 2)

# Normalize the 'cg_communications' column
final['norm_H'] = norm.cdf(final['cg_communications'], mean_H, std_H) * 100

# Remove duplicates based on 'bioguide_id'
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,...,cosponsored-other-party,norm_E,communication_outcome_bipartisanship_sum,norm_F1,communication_outcome_bipartisanship_mean,norm_F2,cg_bonus,norm_G,cg_communications,norm_H
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,...,63.606557,99.907021,197,34.092129,5.98,36.098041,0.0,24.283001,20.0,65.705133
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,...,35.555556,73.760246,274,47.341407,4.83,28.580110,0.0,24.283001,20.0,65.705133
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,...,28.790000,51.548901,185,32.152269,9.74,62.941032,0.0,24.283001,0.0,2.701641
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,...,17.241379,16.342936,160,28.268183,2.49,16.042791,0.0,24.283001,0.0,2.701641
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,...,40.399002,85.621332,164,28.874276,6.65,40.765683,0.0,24.283001,20.0,65.705133


### Source Data 

In [12]:
# Load and filter the source file for communication_attack_personal_sum
source_I1 = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Both-PRL.xlsx')
source_I1 = source_I1[['bioguide_id', 'communication_attack_personal_sum']]

# Merge source_I1 with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_I1, left_on='bioguide_id', right_on='bioguide_id', how='left')

# Fill missing values in 'communication_attack_personal_sum' with the column mean
mean_I1 = round(source_I1['communication_attack_personal_sum'].mean(), 2)
final['communication_attack_personal_sum'] = final['communication_attack_personal_sum'].fillna(mean_I1)

# Calculate mean and standard deviation for normalization
mean_I1 = round(final['communication_attack_personal_sum'].mean(), 2)
std_I1 = round(final['communication_attack_personal_sum'].std(), 2)

# Normalize the 'communication_attack_personal_sum' column
final['norm_I1'] = norm.cdf(final['communication_attack_personal_sum'], mean_I1, std_I1) * 100

# Remove duplicates based on 'bioguide_id'
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,...,communication_outcome_bipartisanship_sum,norm_F1,communication_outcome_bipartisanship_mean,norm_F2,cg_bonus,norm_G,cg_communications,norm_H,communication_attack_personal_sum,norm_I1
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,...,197,34.092129,5.98,36.098041,0.0,24.283001,20.0,65.705133,3,22.852993
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,...,274,47.341407,4.83,28.580110,0.0,24.283001,20.0,65.705133,39,40.429931
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,...,185,32.152269,9.74,62.941032,0.0,24.283001,0.0,2.701641,55,49.227615
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,...,160,28.268183,2.49,16.042791,0.0,24.283001,0.0,2.701641,279,99.903517
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,...,164,28.874276,6.65,40.765683,0.0,24.283001,20.0,65.705133,1,22.018874


### Source Data 

In [13]:
# Load and filter the source file for communication_attack_personal_mean
source_I2 = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Both-PRL.xlsx')
source_I2 = source_I2[['bioguide_id', 'communication_attack_personal_mean']]

# Merge source_I2 with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_I2, left_on='bioguide_id', right_on='bioguide_id', how='left')

# Fill missing values in 'communication_attack_personal_mean' with the column mean
mean_I2 = round(source_I2['communication_attack_personal_mean'].mean(), 2)
final['communication_attack_personal_mean'] = final['communication_attack_personal_mean'].fillna(mean_I2)

# Calculate mean and standard deviation for normalization
mean_I2 = round(final['communication_attack_personal_mean'].mean(), 2)
std_I2 = round(final['communication_attack_personal_mean'].std(), 2)

# Normalize the 'communication_attack_personal_mean' column
final['norm_I2'] = norm.cdf(final['communication_attack_personal_mean'], mean_I2, std_I2) * 100

# Remove duplicates based on 'bioguide_id'
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,...,communication_outcome_bipartisanship_mean,norm_F2,cg_bonus,norm_G,cg_communications,norm_H,communication_attack_personal_sum,norm_I1,communication_attack_personal_mean,norm_I2
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,...,5.98,36.098041,0.0,24.283001,20.0,65.705133,3,22.852993,0.09,17.447710
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,...,4.83,28.580110,0.0,24.283001,20.0,65.705133,39,40.429931,0.69,30.359645
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,...,9.74,62.941032,0.0,24.283001,0.0,2.701641,55,49.227615,2.89,84.971487
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,...,2.49,16.042791,0.0,24.283001,0.0,2.701641,279,99.903517,4.34,98.012503
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,...,6.65,40.765683,0.0,24.283001,20.0,65.705133,1,22.018874,0.04,16.556732


In [14]:
# Load and filter the source file for 'cg_personal_actions' and 'cg_commitments'
source_K = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Senate-CGS-2024.07.xlsx', engine='openpyxl')
source_K = source_K[['bioguide', 'cg_personal_actions', 'cg_commitments']]
print(source_K[source_K['bioguide'] == 'M001153'])
print(source_K[source_K['bioguide'] == 'S001198'])

   bioguide  cg_personal_actions  cg_commitments
55  M001153                 20.0             0.0
   bioguide  cg_personal_actions  cg_commitments
20  S001198                 10.0             0.0


### Source Data 

In [15]:
# Load and filter the source file for 'cg_personal_actions' and 'cg_commitments'
source_K = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Senate-CGS-2024.07.xlsx', engine='openpyxl')
source_K = source_K[['bioguide', 'cg_personal_actions', 'cg_commitments']]

# Combine 'cg_personal_actions' and 'cg_commitments' into a new column 'personal_actions/commitments'
source_K['personal_actions/commitments'] = source_K['cg_personal_actions'] + source_K['cg_commitments']

# Drop the original columns
source_K.drop(['cg_personal_actions', 'cg_commitments'], axis=1, inplace=True)

# Preview the modified source DataFrame
source_K.head()

# Merge source_K with the final DataFrame on 'bioguide_id' and 'biocode'
final = pd.merge(final, source_K, left_on='bioguide_id', right_on='bioguide', how='left')

# Fill missing values in 'personal_actions/commitments' with the column mean
mean_K = round(source_K['personal_actions/commitments'].mean(), 2)
final['personal_actions/commitments'] = final['personal_actions/commitments'].fillna(mean_K)

# Calculate mean and standard deviation for normalization
mean_K = round(final['personal_actions/commitments'].mean(), 2)
std_K = round(final['personal_actions/commitments'].std(), 2)

# Normalize the 'personal_actions/commitments' column
final['norm_K'] = norm.cdf(final['personal_actions/commitments'], mean_K, std_K) * 100

# Drop the 'biocode' column after the merge
final.drop('bioguide', axis=1, inplace=True)

# Remove duplicates based on 'bioguide_id'
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,...,cg_bonus,norm_G,cg_communications,norm_H,communication_attack_personal_sum,norm_I1,communication_attack_personal_mean,norm_I2,personal_actions/commitments,norm_K
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,...,0.0,24.283001,20.0,65.705133,3,22.852993,0.09,17.447710,20.0,83.015856
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,...,0.0,24.283001,20.0,65.705133,39,40.429931,0.69,30.359645,10.0,52.019098
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,...,0.0,24.283001,0.0,2.701641,55,49.227615,2.89,84.971487,0.0,19.668378
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,...,0.0,24.283001,0.0,2.701641,279,99.903517,4.34,98.012503,0.0,19.668378
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,...,0.0,24.283001,20.0,65.705133,1,22.018874,0.04,16.556732,0.0,19.668378


### Source Data 

In [16]:
# Load the source file for 'nominate_dim2' from 'Scores-Both-VoteView.xlsx'
source_L = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Both-VoteView.xlsx')

# Filter columns to keep only 'bioguide_id' and 'nominate_dim2'
source_L = source_L[['bioguide_id', 'nominate_dim2']]

# Preview the filtered source DataFrame
source_L.head()

# Merge source_L with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_L, left_on='bioguide_id', right_on='bioguide_id', how='left')

# Fill missing values in 'nominate_dim2' with the column mean
mean_L = round(source_L['nominate_dim2'].mean(), 2)
final['nominate_dim2'] = final['nominate_dim2'].fillna(mean_L)

# Calculate mean and standard deviation for normalization
mean_L = round(final['nominate_dim2'].mean(), 2)
std_L = round(final['nominate_dim2'].std(), 2)

# Normalize the 'nominate_dim2' column
final['norm_L'] = norm.cdf(final['nominate_dim2'], mean_L, std_L) * 100

# Drop duplicates based on 'bioguide_id' if any exist
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,...,cg_communications,norm_H,communication_attack_personal_sum,norm_I1,communication_attack_personal_mean,norm_I2,personal_actions/commitments,norm_K,nominate_dim2,norm_L
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,...,20.0,65.705133,3,22.852993,0.09,17.447710,20.0,83.015856,-0.314,21.400761
11,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,...,20.0,65.705133,39,40.429931,0.69,30.359645,10.0,52.019098,0.102,77.281460
16,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,...,0.0,2.701641,55,49.227615,2.89,84.971487,0.0,19.668378,0.712,99.868257
17,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,...,0.0,2.701641,279,99.903517,4.34,98.012503,0.0,19.668378,-0.352,17.532394
19,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,...,20.0,65.705133,1,22.018874,0.04,16.556732,0.0,19.668378,0.336,94.682460


### Source Data 

In [17]:
# Load the source file for 'nominate_dim1' from 'Scores-Both-VoteView.xlsx'
source_N = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Both-VoteView.xlsx', engine='openpyxl')

# Filter columns to keep only 'bioguide_id' and 'nominate_dim1'
source_N = source_N[['bioguide_id', 'nominate_dim1']]

# Preview the filtered source DataFrame
source_N.head()

# Merge source_N with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_N, left_on='bioguide_id', right_on='bioguide_id', how='left')

# Fill missing values in 'nominate_dim1' with the column mean
mean_N = round(source_N['nominate_dim1'].mean(), 2)
final['nominate_dim1'] = final['nominate_dim1'].fillna(mean_N)

# Calculate mean and standard deviation for normalization
mean_N = round(final['nominate_dim1'].mean(), 2)
std_N = round(final['nominate_dim1'].std(), 2)

# Normalize the 'nominate_dim1' column
final['norm_N'] = norm.cdf(final['nominate_dim1'], mean_N, std_N) * 100

# Drop duplicates based on 'bioguide_id' if any exist
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,...,communication_attack_personal_sum,norm_I1,communication_attack_personal_mean,norm_I2,personal_actions/commitments,norm_K,nominate_dim2,norm_L,nominate_dim1,norm_N
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,...,3,22.852993,0.09,17.447710,20.0,83.015856,-0.314,21.400761,0.208,67.364478
11,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,...,39,40.429931,0.69,30.359645,10.0,52.019098,0.102,77.281460,0.493,86.383820
16,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,...,55,49.227615,2.89,84.971487,0.0,19.668378,0.712,99.868257,0.571,89.884538
17,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,...,279,99.903517,4.34,98.012503,0.0,19.668378,-0.352,17.532394,0.936,98.233455
19,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,...,1,22.018874,0.04,16.556732,0.0,19.668378,0.336,94.682460,0.427,82.836584


### Source Data 

In [18]:
# Load the source file for 'ideology_intensity' from 'Scores-Both-PRL.xlsx'
source_O = pd.read_excel('/kaggle/input/bridge-pledge-data/Scores-Both-PRL.xlsx', engine='openpyxl')

# Filter the columns to keep only 'bioguide_id' and 'ideology_intensity'
source_O = source_O[['bioguide_id', 'ideology_intensity']]

# Preview the filtered source DataFrame
source_O.head()

# Merge source_O with the final DataFrame on 'bioguide_id'
final = pd.merge(final, source_O, left_on='bioguide_id', right_on='bioguide_id', how='left')

# Fill missing values in 'ideology_intensity' with the column mean
mean_O = round(source_O['ideology_intensity'].mean(), 2)
final['ideology_intensity'] = final['ideology_intensity'].fillna(mean_O)

# Calculate mean and standard deviation for normalization
mean_O = round(final['ideology_intensity'].mean(), 2)
std_O = round(final['ideology_intensity'].std(), 2)

# Normalize the 'ideology_intensity' column
final['norm_O'] = norm.cdf(final['ideology_intensity'], mean_O, std_O) * 100

# Drop duplicates based on 'bioguide_id' if any exist
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,...,communication_attack_personal_mean,norm_I2,personal_actions/commitments,norm_K,nominate_dim2,norm_L,nominate_dim1,norm_N,ideology_intensity,norm_O
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,R,AK,Alaska,not up for election,25.00,...,0.09,17.447710,20.0,83.015856,-0.314,21.400761,0.208,67.364478,0.487516,2.344674
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,R,AK,Alaska,not up for election,27.00,...,0.69,30.359645,10.0,52.019098,0.102,77.281460,0.493,86.383820,1.937050,68.067111
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,R,AL,Alabama,not up for election,28.15,...,2.89,84.971487,0.0,19.668378,0.712,99.868257,0.571,89.884538,2.248260,84.063009
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,R,AL,Alabama,not up for election,2.00,...,4.34,98.012503,0.0,19.668378,-0.352,17.532394,0.936,98.233455,2.561410,93.672058
4,B001236,John,Boozman,John Boozman,Republican,R,AR,Arkansas,not up for election,18.00,...,0.04,16.556732,0.0,19.668378,0.336,94.682460,0.427,82.836584,1.795490,59.081582


### Source Data 

In [19]:
# Load the source file for '2023_pvi_raw' from 'Meta-District-House-PVI.xlsx'
source_M = pd.read_excel('/kaggle/input/bridge-plege-meta-data/Meta-State-PVI.xlsx', engine='openpyxl')

# Filter the columns to keep only 'state', 'PVI party' and '2022 PVI Lean'
source_M = source_M[['State', 'PVI party', '2022 PVI Lean']]

# Preview the filtered source DataFrame
source_M.head()

#Coverting the state and party letter values to lower case and removing the white spaces to get the join to work correctly
final['state'] = final['state'].str.lower().str.strip()
final['party letter'] = final['party letter'].str.lower().str.strip()
source_M['State'] = source_M['State'].str.lower().str.strip()
source_M['PVI party'] = source_M['PVI party'].str.lower().str.strip()

# Merge source_M with the final DataFrame on 'State'and 'Party leter'
final = pd.merge(final, source_M, left_on=['state', 'party letter'], right_on=['State', 'PVI party'], how='left')

# Fill missing values in '2022 PVI Lean' with the column mean
mean_M = round(source_M['2022 PVI Lean'].mean(), 2)
final['2022 PVI Lean'] = final['2022 PVI Lean'].fillna(mean_M)

# Calculate mean and standard deviation for normalization
mean_M = round(final['2022 PVI Lean'].mean(), 2)
std_M = round(final['2022 PVI Lean'].std(), 2)
print(mean_M)
print(std_M)

# Normalize the '2022 PVI Lean' column
final['norm_M'] = norm.cdf(final['2022 PVI Lean'], mean_M, std_M) * 100

# Drop the 'State' and 'PVI party' columns
final.drop('State', axis=1, inplace=True)
final.drop('PVI party', axis=1, inplace=True)

# Drop duplicates based on 'bioguide_id' if any exist
if final['bioguide_id'].duplicated().sum() > 0:
    final.drop_duplicates(subset='bioguide_id', inplace=True)

# Output the preview the updated DataFrame
final.head(5)

10.32
5.26


,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,...,personal_actions/commitments,norm_K,nominate_dim2,norm_L,nominate_dim1,norm_N,ideology_intensity,norm_O,2022 PVI Lean,norm_M
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,r,AK,alaska,not up for election,25.00,...,20.0,83.015856,-0.314,21.400761,0.208,67.364478,0.487516,2.344674,8.0,32.958310
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,r,AK,alaska,not up for election,27.00,...,10.0,52.019098,0.102,77.281460,0.493,86.383820,1.937050,68.067111,8.0,32.958310
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,r,AL,alabama,not up for election,28.15,...,0.0,19.668378,0.712,99.868257,0.571,89.884538,2.248260,84.063009,15.0,81.319559
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,r,AL,alabama,not up for election,2.00,...,0.0,19.668378,-0.352,17.532394,0.936,98.233455,2.561410,93.672058,15.0,81.319559
4,B001236,John,Boozman,John Boozman,Republican,r,AR,arkansas,not up for election,18.00,...,0.0,19.668378,0.336,94.682460,0.427,82.836584,1.795490,59.081582,16.0,85.989504


In [20]:
final.head(10)

,bioguide_id,first name,last name,full name,party name,party letter,district,state,status,bills-with-cosponsor-other-party,...,personal_actions/commitments,norm_K,nominate_dim2,norm_L,nominate_dim1,norm_N,ideology_intensity,norm_O,2022 PVI Lean,norm_M
0,M001153,Lisa,Murkowski,Lisa Murkowski,Republican,r,AK,alaska,not up for election,25.00,...,20.0,83.015856,-0.314,21.400761,0.208,67.364478,0.487516,2.344674,8.00,32.958310
1,S001198,Dan,Sullivan,Dan Sullivan,Republican,r,AK,alaska,not up for election,27.00,...,10.0,52.019098,0.102,77.281460,0.493,86.383820,1.937050,68.067111,8.00,32.958310
2,B001319,Katie Boyd,Britt,Katie Boyd Britt,Republican,r,AL,alabama,not up for election,28.15,...,0.0,19.668378,0.712,99.868257,0.571,89.884538,2.248260,84.063009,15.00,81.319559
3,T000278,Tommy,Tuberville,Tommy Tuberville,Republican,r,AL,alabama,not up for election,2.00,...,0.0,19.668378,-0.352,17.532394,0.936,98.233455,2.561410,93.672058,15.00,81.319559
4,B001236,John,Boozman,John Boozman,Republican,r,AR,arkansas,not up for election,18.00,...,0.0,19.668378,0.336,94.682460,0.427,82.836584,1.795490,59.081582,16.00,85.989504
5,C001095,Tom,Cotton,Tom Cotton,Republican,r,AR,arkansas,not up for election,8.00,...,10.0,52.019098,0.178,84.840805,0.583,90.358841,2.036000,73.803141,16.00,85.989504
6,S001191,Kyrsten,Sinema,Kyrsten Sinema,Independent,i,AZ,arizona,retiring,29.00,...,40.0,99.713733,0.076,74.275163,-0.105,39.690605,0.586884,3.446787,10.02,47.725897
7,K000377,Mark,Kelly,Mark Kelly,Democratic,d,AZ,arizona,not up for election,21.00,...,10.0,52.019098,0.255,90.571393,-0.217,30.295895,1.119180,17.966459,10.02,47.725897
8,P000145,Alex,Padilla,Alex Padilla,Democratic,d,CA,california,not up for election,15.00,...,0.0,19.668378,-0.331,19.612183,-0.375,19.078695,1.488930,38.592757,13.00,69.480110
9,B001320,Laphonza R.,Butler,Laphonza R. Butler,Democratic,d,CA,california,retiring,28.15,...,0.0,19.668378,0.056,71.829292,-0.542,10.482191,1.473120,37.571826,13.00,69.480110


### Adding Weights

In [21]:
weights = {
    'A': 1,
    'B': 0.5,
    'D': 1.5,
    'E': 1,
    'F1': 0.75,
    'F2': 0.75,
    'G': 0.5,
    'H': 0.5,
    'I1': 0.75,
    'I2': 0.75,
    'K': 0.5,
    'L': 0.5,
    'M': 0.001,
    'N': 0.0005,
    'O': 0.0005
}

print(weights)


{'A': 1, 'B': 0.5, 'D': 1.5, 'E': 1, 'F1': 0.75, 'F2': 0.75, 'G': 0.5, 'H': 0.5, 'I1': 0.75, 'I2': 0.75, 'K': 0.5, 'L': 0.5, 'M': 0.001, 'N': 0.0005, 'O': 0.0005}


### Score Calculation & Final Score

In [22]:
def cal_score(data, weights):
    # Copy the data to avoid modifying the original dataframe
    temp_data = data.copy()
    
    # Initialize columns
    temp_data['score_T'] = 0
    
    # Calculate 'score_T' by adding weighted norm values for each category
    for i in weights.keys():
        temp_data['score_T'] += temp_data[f"norm_{i}"] * weights[i]
    
    # Calculate 'score_S' by adjusting 'score_T' with specific terms based on weights
    temp_data['score_S'] = temp_data['score_T'] * (1 + ((temp_data['norm_M'] * weights['M']) +
                                                       (temp_data['norm_N'] * weights['N']) +
                                                       (temp_data['norm_O'] * weights['O'])))
    
    # Calculate mean, std, and median of 'score_S'
    mean_S = round(temp_data['score_S'].mean(), 2)
    std_S = round(temp_data['score_S'].std(), 2)
    median_S = round(temp_data['score_S'].median(), 2)
    
    # Normalize 'score_S' using the cumulative distribution function (CDF)
    temp_data['norm_S'] = norm.cdf(temp_data['score_S'], mean_S, std_S) * 100
    
    # Define the grade assignment function based on the calculated scores
    def assign_grade(grade):
        if grade > mean_S + std_S:
            return 'A'
        elif grade > median_S:
            return 'B'
        elif grade > mean_S - std_S:
            return 'C'
        else:
            return 'F'
    
    # Apply the grade assignment to the 'score_S' column
    temp_data['Grade'] = temp_data['score_S'].apply(assign_grade)
    
    return temp_data

# Call the function to get the final scores and grades
scores = cal_score(final, weights)
# Display the specific columns
scores[['bioguide_id', 'full name', 'party name', 'score_S', 'Grade']].head(21)

,bioguide_id,full name,party name,score_S,Grade
0,M001153,Lisa Murkowski,Republican,551.475343,B
1,S001198,Dan Sullivan,Republican,476.753545,B
2,B001319,Katie Boyd Britt,Republican,420.190707,C
3,T000278,Tommy Tuberville,Republican,307.708444,F
4,B001236,John Boozman,Republican,529.750188,B
5,C001095,Tom Cotton,Republican,382.143105,C
6,S001191,Kyrsten Sinema,Independent,652.459309,A
7,K000377,Mark Kelly,Democratic,536.512696,B
8,P000145,Alex Padilla,Democratic,301.590420,F
9,B001320,Laphonza R. Butler,Democratic,423.842590,C
